In [11]:
import stockutils
import pandas_datareader.data as web, re, sqlite3, datetime

In [57]:
class StockData:
    def __init__(self, se = None, update = False, force_update = False):
        self.company_table = stockutils.company_table(se = se, update = update, force_update = force_update)
        self.se = se
   
    def lookup_ticker(self, company_name, sql = True):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        if isinstance(company_name,str):
            company_name = [company_name]
        result_table, conn = [], None 
        
        if not sql:
            for name in company_name:
                name = r'.*\s*'+name.lower()+r'.*'
                for row in self.company_table:
                    match = re.search(name,row[1].lower()) 
                    if match:
                        result_table.append((row[0],row[1],row[9]))
        elif self.se:
            conn = sqlite3.connect('data/companysql' + self.se + '.sql')
        else:
            conn = sqlite3.connect('data/completecompanylist.sql')
        
        if conn: 
            c = conn.cursor()
            for name in company_name:
                query = ('%'+name+'%',)
                rows = c.execute('SELECT symbol,name,se FROM companytable WHERE name LIKE ?;', query).fetchall()
                if rows:
                    for row in rows:
                        result_table.append(row)
                else:
                    print('Switching to table for ' + name + ' lookup.')
                    result_table.append(self.lookup_ticker(name, sql = False))
            conn.close()
        if result_table:
            return result_table.pop() if len(result_table) == 1 else result_table
        else:
            return ('N/A', company_name)
    
#    def stock_price(self, ticker, date):
        
        

In [58]:
w = StockData()

In [59]:
x = StockData('nasdaq')

In [60]:
y = StockData('amex')

In [61]:
z = StockData('nyse')

In [62]:
x.lookup_ticker('Nvidia')

('NVDA', 'NVIDIA Corporation', 'nasdaq')

In [63]:
x.lookup_ticker(['Intel Corporation', 'Nvidia', 'Apple', 'Microsoft', 'Becton Dickinson'])

Switching to table for Becton Dickinson lookup.


[('INTC', 'Intel Corporation', 'nasdaq'),
 ('NVDA', 'NVIDIA Corporation', 'nasdaq'),
 ('AAPL', 'Apple Inc.', 'nasdaq'),
 ('MSFT', 'Microsoft Corporation', 'nasdaq'),
 ('N/A', ['Becton Dickinson'])]

In [64]:
z.lookup_ticker("Dick's")

('DKS', "Dick's Sporting Goods Inc", 'nyse')

In [65]:
z.lookup_ticker('Becton')

('BDX', 'Becton, Dickinson and Company', 'nyse')

In [68]:
w.lookup_ticker('Intel')

[('BOTZ',
  'Global X Robotics & Artificial Intelligence Thematic ETF',
  'nasdaq'),
 ('INTC', 'Intel Corporation', 'nasdaq'),
 ('IQNT', 'Inteliquent, Inc.', 'nasdaq'),
 ('NTLA', 'Intellia Therapeutics, Inc.', 'nasdaq'),
 ('IPCI', 'Intellipharmaceutics International Inc.', 'nasdaq'),
 ('GOL', 'Gol Linhas Aereas Inteligentes S.A.', 'nyse'),
 ('I', 'Intelsat S.A.', 'nyse'),
 ('IDN', 'Intellicheck Mobilisa, Inc.', 'amex'),
 ('INS', 'Intelligent Systems Corporation', 'amex')]